In [1]:
import pandas as pd

In [2]:
#leitura das duas abas do excel, defini client com str na leitura, visto q é um id
path = './../Base-Dados-Desafio-500k.xlsx'
df_sales = pd.read_excel(
    path, 
    sheet_name='VENDAS',
    dtype={'CLIENTE': str}
)

df_categories = pd.read_excel(path, sheet_name='PRODUTOS')

In [3]:
#aqui é possível observar os valores nulos que serão tratados adiante e os tipos
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CLIENTE             499484 non-null  object        
 1   IDADE               499850 non-null  float64       
 2   ESTADO              499630 non-null  object        
 3   PRODUTO             499452 non-null  object        
 4   QUANTIDADE_VENDIDA  500000 non-null  int64         
 5   PREÇO_UNITARIO      500000 non-null  int64         
 6   DATA                500000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 26.7+ MB


In [4]:
num_columns = ['IDADE', 'QUANTIDADE_VENDIDA', 'PREÇO_UNITARIO']
df_sales[num_columns].describe()

,IDADE,QUANTIDADE_VENDIDA,PREÇO_UNITARIO
count,499850.000000,500000.000000,500000.000000
mean,39.183431,185.928544,32.980870
std,11.560814,80.424516,12.914743
min,18.000000,26.000000,0.000000
25%,31.000000,147.000000,25.000000
50%,39.000000,187.000000,30.000000
75%,47.000000,231.000000,45.000000
max,80.000000,432.000000,50.000000


In [5]:
df = df_sales.merge(df_categories, how="left", on="PRODUTO")

df["FATURAMENTO"] = (
    df["QUANTIDADE_VENDIDA"] * df["PREÇO_UNITARIO"]
)

df["DATA"] = pd.to_datetime(df["DATA"])
df["ANO"] = df["DATA"].dt.year
df["MES"] = df["DATA"].dt.month
df["DIA"] = df["DATA"].dt.day

#tratamento de valores nulos
#para id foi preenchido 0 visto q a análise em si não depende do valor do id
#para idades foi usada a mediana, por ser uma medida menos sensível à outliers
#já paras as categorias restantes, 'Indefinido'
df['CLIENTE'] = df['CLIENTE'].fillna('0')
median_age = df['IDADE'].median()
df["IDADE"] = df["IDADE"].fillna(median_age)
df["ESTADO"] = df["ESTADO"].fillna("Indefinido")
df["PRODUTO"] = df["PRODUTO"].fillna("Indefinido")
df["CATEGORIA"] = df["CATEGORIA"].fillna("Indefinido")

df["FAIXA_ETARIA"] = pd.cut(
    df["IDADE"],
    bins=[0, 24, 34, 44, 54, 200],
    labels=["18–24", "25–34", "35–44", "45–54", "55+"],
    right=True
)

df.head()

,CLIENTE,IDADE,ESTADO,PRODUTO,QUANTIDADE_VENDIDA,PREÇO_UNITARIO,DATA,CATEGORIA,FATURAMENTO,ANO,MES,DIA,FAIXA_ETARIA
0,9010,72.0,SANTA CATARINA,E,243,50,2024-10-26,JARDINAGEM,12150,2024,10,26,55+
1,63611,46.0,PARANÁ,B,320,15,2024-12-27,MANGUEIRAS,4800,2024,12,27,45–54
2,56708,67.0,BAHIA,D,143,30,2023-06-30,LIMPEZA,4290,2023,6,30,55+
3,3924,46.0,SÃO PAULO,B,295,15,2025-06-02,MANGUEIRAS,4425,2025,6,2,45–54
4,47317,53.0,RIO DE JANEIRO,C,203,45,2025-02-15,UTILIDADES DOMÉSTICAS,9135,2025,2,15,45–54


In [6]:
#foi salvo em parquet devido a utilização de metadados e facilidade de uso entre diversos locais
#ainda seria possível fazer algum particionamento ex: por produto, para melhor eficiência de leitura
df.to_parquet('./../data/sales.parquet')